<a href="https://colab.research.google.com/github/RockNavamuel/uber_lyft_viz/blob/main/Data_Viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown ![](https://drive.google.com/uc?id=16xE3KeHzbt5lYVHp5Gh5puiP5ufqm8sA)

In [ ]:
import google.cloud.bigquery as bigquery

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
client = bigquery.Client(project='your-database-id')

In [ ]:
# Set use_legacy_sql to True to use legacy SQL syntax.
job_config = bigquery.QueryJobConfig(use_legacy_sql=True)

In [ ]:
query_string = ("""

SELECT *
FROM
  Uber_Lyft.pricing
ORDER BY
  cab_type
""")

df = client.query(query_string, job_config=job_config).to_dataframe()
df.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,2.0,Lyft,1.540000e+12,Theatre District,Northeastern University,9.0,1.0,31d5a99d-a604-4d05-bd8b-399d29717fc3,lyft,Lyft
1,2.0,Lyft,1.540000e+12,Theatre District,Northeastern University,9.0,1.0,f385c5ca-f76c-4a53-a7e3-dd1d201addbc,lyft,Lyft
2,2.0,Lyft,1.540000e+12,Theatre District,Northeastern University,9.0,1.0,12165cd6-3b57-4a8e-96a6-4850782d3f70,lyft,Lyft
3,2.0,Lyft,1.540000e+12,Back Bay,Boston University,9.0,1.0,47ae11d3-f052-413c-9548-f7b0b0c36024,lyft,Lyft
4,2.0,Lyft,1.540000e+12,Theatre District,Northeastern University,9.0,1.0,de7c81c4-0e10-4d4b-a35e-f6a77025a290,lyft,Lyft


**Visualization 1:**
Over the past 5 years, Lyft has received 4x less Google searches, on average, than competitor Uber

In [ ]:
query_string1 = ("""
SELECT
  ROUND(AVG(Lyft)) as Lyft, ROUND(AVG(Uber)) as Uber
FROM
  Uber_Lyft.trend
""")

df1 = client.query(query_string1, job_config=job_config).to_dataframe()
df1

,Lyft,Uber
0,8.0,36.0


**Visualization 2:** Top Destinations for Uber and Lyft

In [ ]:
query_string2 = ("""
SELECT
  destination,
  MAX(CASE WHEN cab_type = 'Lyft' THEN orders END) AS Lyft,
  MAX(CASE WHEN cab_type = 'Uber' THEN orders END) AS Uber
FROM
  (SELECT
      destination,
      cab_type,
      COUNT(cab_type) AS orders
  FROM
      Uber_Lyft.pricing
  GROUP BY
      destination,
      cab_type) subquery
GROUP BY
  destination
ORDER BY
  destination
""")

df2 = client.query(query_string2, job_config=job_config).to_dataframe()
df2

,destination,Lyft,Uber
0,Back Bay,25650,32130
1,Beacon Hill,25464,31939
2,Boston University,25614,32150
3,Fenway,25620,32137
4,Financial District,26238,32613
5,Haymarket Square,25613,32151
6,North End,25614,32142
7,North Station,25326,31793
8,Northeastern University,25619,32136
9,South Station,25620,32129


In [ ]:
df2.to_csv('/content/drive/My Drive/top_destinations.csv', index=False)

**Visualization 3**: Price differences

In [ ]:
query_string3 = """
SELECT
  distance, cab_type, price
FROM
  Uber_Lyft.pricing
WHERE price IS NOT NULL
"""
df3 = client.query(query_string3, job_config=job_config).to_dataframe()
df3

,distance,cab_type,price
0,2.00,Lyft,3.0
1,2.00,Lyft,3.0
2,2.00,Lyft,3.0
3,2.00,Lyft,3.0
4,2.00,Lyft,3.0
...,...,...,...
637971,1.81,Uber,30.5
637972,1.81,Uber,31.5
637973,1.81,Uber,31.5
637974,1.81,Uber,31.5


In [ ]:
# Create sample of data for clearer visualization
import pandas as pd
total_rows = df3.shape[0]
sample_size = 1000

df_lyft = df3[df3['cab_type'] == 'Lyft']
df_uber = df3[df3['cab_type'] == 'Uber']

# Divide by 2 since we want an equal number for Lyft and Uber
sample_size_per_cab_type = sample_size // 2

df_lyft_sampled = df_lyft.sample(n=sample_size_per_cab_type, random_state=4)
df_uber_sampled = df_uber.sample(n=sample_size_per_cab_type, random_state=4)

# Concatenate the sampled DataFrames back together
df3_sampled = pd.concat([df_lyft_sampled, df_uber_sampled])
df3_sampled = df3_sampled.sample(frac=1, random_state=4)

df3_sampled.head()

,distance,cab_type,price
471362,2.84,Uber,25.0
190879,1.58,Uber,13.0
457458,1.23,Uber,16.0
237462,0.98,Uber,27.5
87607,0.91,Uber,15.0


In [ ]:
df3_sampled.to_csv('/content/drive/My Drive/distance_pricing_sampled3.csv', index=False)

**Visualization 4:** Distribution of Pricing (pie chart)

In [ ]:
query_string4 = """
SELECT
  cab_type, ROUND(AVG(price),2) as avg_price, name
FROM
  Uber_Lyft.pricing
WHERE
  name in ('Lux Black XL', 'Black SUV', 'Lux Black', 'Black',
          'Lyft XL', 'UberXL', 'Lyft', 'UberX', 'Shared', 'UberPool')
GROUP BY cab_type, name
ORDER BY avg_price
"""
df4 = client.query(query_string4, job_config=job_config).to_dataframe()
df4

,cab_type,avg_price,name
0,Lyft,6.03,Shared
1,Uber,8.75,UberPool
2,Lyft,9.61,Lyft
3,Uber,9.77,UberX
4,Lyft,15.31,Lyft XL
5,Uber,15.68,UberXL
6,Uber,20.52,Black
7,Lyft,23.06,Lux Black
8,Uber,30.29,Black SUV
9,Lyft,32.32,Lux Black XL


In [ ]:
df4.to_csv('/content/drive/My Drive/avg_prices.csv', index=False)

**Visualization 5:** Surge pricing

In [ ]:
query_string5 = """
SELECT
  surge_multiplier, cab_type, price
FROM
  Uber_Lyft.pricing
WHERE price IS NOT NULL AND cab_type = 'Lyft'
"""
df5 = client.query(query_string5, job_config=job_config).to_dataframe()
df5

,surge_multiplier,cab_type,price
0,2.00,Lyft,10.5
1,2.00,Lyft,10.5
2,2.00,Lyft,10.5
3,2.00,Lyft,10.5
4,2.00,Lyft,10.5
...,...,...,...
307403,1.75,Lyft,67.5
307404,1.75,Lyft,67.5
307405,1.75,Lyft,67.5
307406,1.75,Lyft,67.5


In [ ]:
df5.to_csv('/content/drive/My Drive/surge_pricing.csv', index=False)